<a href="https://colab.research.google.com/github/LourdesNajarro/Lab2-SP1/blob/main/SP1_Laboratory_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PERSONAL INFORMATION

Name: Lourdes María Najarro Herrera

ID: 22000795

# Requirements:

*   Implement the whole ML Pipeline (Acquire data, pre-process data, create model, train model & evaluate model)
*   Accuracy must be above 40%
*   Must use the Tensorboard and display the model's train process
*   Must use the following Callbacks: EarlyStopping and ModelCheckpoint
*   Extra (+10): Include confusion Matrix and 2 suggestions on how to improve the model (just write them, do not implement them)




## Preparations

In [48]:
# ----- Libraries ----- #

# This is the main Library that allows us to work with Neural Networks
import tensorflow as tf


# For graph plotting
import matplotlib.pyplot as plt
from tensorflow.math import confusion_matrix

# For dataset manipulation
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# For visualizing more complex graphs
import seaborn as sns

# Global constant for training acceleration
AUTOTUNE = tf.data.AUTOTUNE

# 1) Dataset Preparations

In [20]:
'''
The dataset you are going to use is the following:
https://archive.ics.uci.edu/ml/datasets/Credit+Approval
'''

# Getting Dataset
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"

--2025-03-09 15:00:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘crx.data’

crx.data                [ <=>                ]  31.46K  --.-KB/s    in 0.06s   

2025-03-09 15:00:02 (485 KB/s) - ‘crx.data’ saved [32218]



In [49]:
##leemos el archivo y data
columnas = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']

data = pd.read_csv('crx.data', names=columnas, na_values='?', comment='\t', sep=',', skipinitialspace=True)

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [50]:
##PRE PROCESAR LA DATA
data_copia = data.copy()

# eliminar los missing values
data_copia = data_copia.dropna()

#observar el dataset
data_copia

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


In [51]:
##los categoricos de dos opciones cambiarlos a cero o uno
from sklearn.preprocessing import LabelEncoder

# Aplicar Label Encoding a las columnas con solo 2 valores o muchas categorías
label_cols = ["A1", "A6", "A9", "A10", "A12", "A16"]
label_encoders = {}

for col in label_cols:
    le = LabelEncoder()
    data_copia[col] = le.fit_transform(data_copia[col])
    label_encoders[col] = le  # Guardar encoders para futura decodificación

In [44]:
data_copia

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,30.83,0.000,u,g,12,v,1.25,1,1,1,0,g,202.0,0,0
1,0,58.67,4.460,u,g,10,h,3.04,1,1,6,0,g,43.0,560,0
2,0,24.50,0.500,u,g,10,h,1.50,1,0,0,0,g,280.0,824,0
3,1,27.83,1.540,u,g,12,v,3.75,1,1,5,1,g,100.0,3,0
4,1,20.17,5.625,u,g,12,v,1.71,1,0,0,0,s,120.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21.08,10.085,y,p,4,h,1.25,0,0,0,0,g,260.0,0,1
686,0,22.67,0.750,u,g,1,v,2.00,0,1,2,1,g,200.0,394,1
687,0,25.25,13.500,y,p,5,ff,2.00,0,1,1,1,g,200.0,1,1
688,1,17.92,0.205,u,g,0,v,0.04,0,0,0,0,g,280.0,750,1


In [52]:
### one hot enconding para las columnas
#con mas de dos valores, pero que no sean de muchos valores categoricos
data_copia = pd.get_dummies(data_copia, columns=["A4", "A5", "A7", "A13"], drop_first=False).astype(int)

In [53]:
data_copia

,A1,A2,A3,A6,A8,A9,A10,A11,A12,A14,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
0,1,30,0,12,1,1,1,1,0,202,...,0,0,0,0,0,1,0,1,0,0
1,0,58,4,10,3,1,1,6,0,43,...,0,1,0,0,0,0,0,1,0,0
2,0,24,0,10,1,1,0,0,0,280,...,0,1,0,0,0,0,0,1,0,0
3,1,27,1,12,3,1,1,5,1,100,...,0,0,0,0,0,1,0,1,0,0
4,1,20,5,12,1,1,0,0,0,120,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21,10,4,1,0,0,0,0,260,...,0,1,0,0,0,0,0,1,0,0
686,0,22,0,1,2,0,1,2,1,200,...,0,0,0,0,0,1,0,1,0,0
687,0,25,13,5,2,0,1,1,1,200,...,1,0,0,0,0,0,0,1,0,0
688,1,17,0,0,0,0,0,0,0,280,...,0,0,0,0,0,1,0,1,0,0


# READ ME

When you are finished with your laboratory, remember to share it. Make the URL public for Galileo.edu domain or for everyone.
Also, remember to hit Ctrl+S to save your progress.